# Downloading IMERG data for nowcasting

This notebook shows how to use the nowcasting library code in order to download IMERG events. Specifically, we provide examples for downloading individual events as well as entire datasets to be used any downstream tasks. 


Author: Akshay Aravamudan  (aaravamudan2014@my.fit.edu)

Last edited: Jan 21, 2025 

## Imports


In [1]:
from servir.utils.download_data import get_event, get_gpm_files, initialize_event_folders
from servir.utils.m_tif2h5py import tif2h5py

# get path of nowcasting folder
nowcasting_path = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting'

# create data folder if it does not exist
from pathlib import Path
output_file = Path(nowcasting_path + '/data/flood_events_wa.csv')
output_file.parent.mkdir(exist_ok=True, parents=True)




## Downloading specific events

This example assumes that there is a csv file containing some events with the following columns

1. Event ID
2. Region in Ghana
3. Event  Recorded Date
4. Coordinates (optional)
5. link (optional)

For instance it could be a csv file with the following data,
|Event ID|Region in Ghana                                                                      |Event  Recorded Date|Coordinates             |link                                                                                          |
|--------|-------------------------------------------------------------------------------------|-------------------|------------------------|----------------------------------------------------------------------------------------------|
|1       |Bunkpurugu                                                                           |August 30, 2013    |10°30′45.72″N 0°5′52.8″E|https://www.modernghana.com/news/486439/1/bunkpurugu-floods-one-dead-about-6000-displaced.html|
|2       |Accra                                                                                |January 31, 2014   |5°33′N 0°12′W           |https://floodlist.com/africa/ghana-accra-hit-flash-floods                                     |
|3       |Accra                                                                                |June 5, 2014       |5°33′N 0°12′W           |https://floodlist.com/africa/accra-ghana-floods-june-2014                                     |
|4       |Accra                                                                                |July 10, 2014      |5°33′N 0°12′W           |https://floodlist.com/africa/1-dead-floods-jomoro-ghana                                       |
|5       |Kumasi, Axim and Accra                                                               |June 3, 2015       |06°42′00″N 01°37′30″W   |https://floodlist.com/africa/flood-accra-ghana-fire-explosion                                 |




### Obtain timestamps

In [2]:
# let us pick a particular event to download

event_id = 1
initial_timestamp, final_timestamp = get_event(csv_filename= nowcasting_path + '/data/flood_events_wa.csv', event_id=event_id)


### Set data parameters

In [3]:
domain_location = 'WA'

# Domain coordinates for WA 
xmin = -21.4
xmax = 30.4
ymin = -2.9
ymax = 33.1

# Domain coordinates for Ghana 
# xmin = -3.95
# xmax = 2.35
# ymin = 4.85
# ymax = 11.15


# The outputs would assume the following shape
req_shape = (360, 518)

precipBaseFolder = nowcasting_path + '/data/events/'


#  the following cell creates folders if not already created (assuming that the preceding directories already exist)
precipEventFolder = initialize_event_folders(precipBaseFolder, event_id=event_id)

# server path for where to download IMERG data from 
ppt_server_path = 'https://jsimpsonhttps.pps.eosdis.nasa.gov/imerg/gis/early/'

# email associated with accound accessing the NASA product
# this is the authors email and is free to use by other members
email = 'aaravamudan2014@my.fit.edu'

### Download the data and store it in relevant format


In [4]:
# the following function will download the image and store it in `precipEventFolder`
# note that it will ignore if the file already exists in the folder

req_height, req_width = req_shape
get_gpm_files(precipEventFolder,initial_timestamp, final_timestamp, ppt_server_path ,email, req_height, req_width)


<class 'datetime.datetime'>


In [5]:
# once the files are downloaded in the relevant folder, we need to convert it into h5 format
# since it is the format that the library uses for processing data

# this is location of the metadata folder which contains all the coordinate information that
# will be used for visualization purposes
meta_fname = precipEventFolder + 'metadata.json'

tif2h5py(tif_directory=precipEventFolder, h5_fname=precipBaseFolder + str(event_id)+'.h5', meta_fname=meta_fname, x1=xmin, y1=ymin, x2=xmax, y2=ymax)


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [6]:
print(meta_fname)

/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json
